<h1> Text Classification using TensorFlow/Keras on Cloud ML Engine </h1>

This notebook illustrates:
<ol>
<li> Creating datasets for Machine Learning using BigQuery
<li> Creating a text classification model using the Estimator API with a Keras model
<li> Training on Cloud ML Engine
<li> Deploying the model
<li> Predicting with model
<li> Rerun with pre-trained embedding
</ol>

In [1]:
# change these to try this notebook out
BUCKET = 'vijays-sandbox-ml'
PROJECT = 'vijays-sandbox'
REGION = 'us-central1'
SEED = 0

In [2]:
import tensorflow as tf
print(tf.__version__) # tf 2.0 nightly
print(tf.test.is_gpu_available())

2.0.0-beta1
True


# Pure Keras

In [11]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np
import re
import pickle

from tensorflow.python.keras.preprocessing import sequence
from tensorflow.python.keras.preprocessing import text
from tensorflow.python.keras import models
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.layers import Dropout
from tensorflow.python.keras.layers import Embedding
from tensorflow.python.keras.layers import Conv1D
from tensorflow.python.keras.layers import MaxPooling1D
from tensorflow.python.keras.layers import GlobalAveragePooling1D

from google.cloud import storage

CLASSES = {'github': 0, 'nytimes': 1, 'techcrunch': 2}  # label-to-int mapping
TOP_K = 20000  # Limit on the number vocabulary size used for tokenization
MAX_SEQUENCE_LENGTH = 50  # Sentences will be truncated/padded to this length

"""
Helper function to download data from Google Cloud Storage
  # Arguments:
      source: string, the GCS URL to download from (e.g. 'gs://bucket/file.csv')
      destination: string, the filename to save as on local disk. MUST be filename
      ONLY, doesn't support folders. (e.g. 'file.csv', NOT 'folder/file.csv')
  # Returns: nothing, downloads file to local disk
"""
def download_from_gcs(source, destination):
    search = re.search('gs://(.*?)/(.*)', source)
    bucket_name = search.group(1)
    blob_name = search.group(2)
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    bucket.blob(blob_name).download_to_filename(destination)


"""
Parses raw tsv containing hacker news headlines and returns (sentence, integer label) pairs
  # Arguments:
      train_data_path: string, path to tsv containing training data.
        can be a local path or a GCS url (gs://...)
      eval_data_path: string, path to tsv containing eval data.
        can be a local path or a GCS url (gs://...)
  # Returns:
      ((train_sentences, train_labels), (test_sentences, test_labels)):  sentences
        are lists of strings, labels are numpy integer arrays
"""
def load_hacker_news_data(train_data_path, eval_data_path):
    if train_data_path.startswith('gs://'):
        download_from_gcs(train_data_path, destination='train.csv')
        train_data_path = 'train.csv'
    if eval_data_path.startswith('gs://'):
        download_from_gcs(eval_data_path, destination='eval.csv')
        eval_data_path = 'eval.csv'

    # Parse CSV using pandas
    column_names = ('label', 'text')
    df_train = pd.read_csv(train_data_path, names=column_names, sep='\t')
    df_eval = pd.read_csv(eval_data_path, names=column_names, sep='\t')

    return ((list(df_train['text']), np.array(df_train['label'].map(CLASSES))),
            (list(df_eval['text']), np.array(df_eval['label'].map(CLASSES))))


"""
Create tf.estimator compatible input function
  # Arguments:
      texts: [strings], list of sentences
      labels: numpy int vector, integer labels for sentences
      tokenizer: tf.python.keras.preprocessing.text.Tokenizer
        used to convert sentences to integers
      batch_size: int, number of records to use for each train batch
      mode: tf.estimator.ModeKeys.TRAIN or tf.estimator.ModeKeys.EVAL 
  # Returns:
      tf.data.dataset, produces feature and label
        tensors one batch at a time
"""
def input_fn(texts, labels, batch_size, mode):
    def _embed(sentence,label):
        #embeddings = embed(sentence)
        #return embeddings,label
        return sentence,label
    
    #precision and recall metrics require one hot labels
    labels = tf.one_hot(labels,len(CLASSES)) 
    #split sentences into lists of words
    texts = [sentence.split() for sentence in texts] 
    # pad to constant length
    texts = [(sentence + MAX_SEQUENCE_LENGTH * ['<PAD>'])[:MAX_SEQUENCE_LENGTH] for sentence in texts] 
    #embed
    embed = hub.load("https://tfhub.dev/google/tf2-preview/nnlm-en-dim128-with-normalization/1")
    texts = [embed(sentence) for sentence in texts]

    dataset = tf.data.Dataset.from_tensor_slices((texts, labels))
    
    if mode == tf.estimator.ModeKeys.EVAL:
        return dataset.batch(batch_size)
    else: 
        return dataset.shuffle(50000).batch(batch_size)

"""
Builds a CNN model using keras and converts to tf.estimator.Estimator
  # Arguments
      model_dir: string, file path where training files will be written
      config: tf.estimator.RunConfig, specifies properties of tf Estimator
      filters: int, output dimension of the layers.
      kernel_size: int, length of the convolution window.
      embedding_dim: int, dimension of the embedding vectors.
      dropout_rate: float, percentage of input to drop at Dropout layers.
      pool_size: int, factor by which to downscale input at MaxPooling layer.
      embedding_path: string , file location of pre-trained embedding (if used)
        defaults to None which will cause the model to train embedding from scratch
      word_index: dictionary, mapping of vocabulary to integers. used only if
        pre-trained embedding is provided

    # Returns
        A tf.estimator.Estimator 
"""
def keras_estimator(model_dir,
                    config,
                    learning_rate,
                    filters=64,
                    dropout_rate=0.2,
                    embedding_dim=200,
                    kernel_size=3,
                    pool_size=3,
                    embedding_path=None,
                    word_index=None):
    # Create model instance.
    model = models.Sequential()

    #model.add(tf.keras.layers.InputLayer(input_shape=(MAX_SEQUENCE_LENGTH,128)))
    model.add(Dropout(input_shape=(MAX_SEQUENCE_LENGTH,128),rate=dropout_rate))
    model.add(Conv1D(
        #input_shape=(MAX_SEQUENCE_LENGTH,128),
        filters=filters,
        kernel_size=kernel_size,
        activation='relu',
        bias_initializer='random_uniform',
        padding='same'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Conv1D(
        filters=filters * 2,
        kernel_size=kernel_size,
        activation='relu',
        bias_initializer='random_uniform',
        padding='same'))
    model.add(GlobalAveragePooling1D())
    model.add(Dropout(rate=dropout_rate))
    model.add(Dense(len(CLASSES), activation='softmax'))

    # Compile model with learning parameters.
    optimizer = tf.keras.optimizers.Adam(lr=learning_rate)
    model.compile(
        optimizer=optimizer, 
        loss='categorical_crossentropy', 
        metrics=[
            'accuracy',
            tf.keras.metrics.Precision(),
            tf.keras.metrics.Recall()
        ]
    )

    return model


"""
Defines the features to be passed to the model during inference 
  Expects already tokenized and padded representation of sentences
  # Arguments: none
  # Returns: tf.estimator.export.ServingInputReceiver
"""
def serving_input_fn():
    feature_placeholder = tf.compat.v1.placeholder(tf.string, [None])
    features = feature_placeholder  # pass as-is
    return tf.estimator.export.TensorServingInputReceiver(features, feature_placeholder)

In [4]:
hparams = {'train_data_path':'./data/txtcls/train.tsv',
           'eval_data_path':'./data/txtcls/eval.tsv'}

# Load Data
((train_texts, train_labels), (test_texts, test_labels)) = load_hacker_news_data(
    hparams['train_data_path'], hparams['eval_data_path'])

In [5]:
%%time
dataset = input_fn(
    test_texts,
    test_labels,
    3,
    mode=tf.estimator.ModeKeys.EVAL
)
for batch in dataset:
    print(batch)
    break

(<tf.Tensor: id=48280, shape=(3, 50, 128), dtype=float32, numpy=
array([[[ 0.05921581,  0.2607566 , -0.01803753, ..., -0.09629437,
          0.1261976 , -0.10059416],
        [-0.05234646, -0.04587635,  0.09989155, ...,  0.10251415,
         -0.04885277, -0.02132634],
        [-0.23469844,  0.13233985,  0.15608735, ...,  0.13845624,
         -0.14491926,  0.08018108],
        ...,
        [-0.24159245,  0.06395964, -0.07213151, ...,  0.24148819,
         -0.09287039,  0.11624853],
        [-0.24159245,  0.06395964, -0.07213151, ...,  0.24148819,
         -0.09287039,  0.11624853],
        [-0.24159245,  0.06395964, -0.07213151, ...,  0.24148819,
         -0.09287039,  0.11624853]],

       [[-0.08414377,  0.00414404,  0.06067044, ...,  0.04296787,
         -0.09396288, -0.15785182],
        [-0.04778689,  0.10285304,  0.04732528, ..., -0.10652717,
         -0.01119212, -0.0911274 ],
        [ 0.23039751, -0.12019835, -0.03054287, ...,  0.07649281,
         -0.08489004, -0.01898622],
  

In [6]:
test_texts[:5]

['show hn  scrwl   shorthand code reading and writing language',
 'geoip module on nodejs now is a c   addon',
 'vilfredo pareto on software product management',
 ' walking around the earth   street view virtual walking on google earth',
 'projector - easy mockup presentation engine']

In [7]:
%%time
hparams = {'batch_size':128}

train_dataset = input_fn(
    train_texts,
    train_labels,
    hparams['batch_size'],
    mode=tf.estimator.ModeKeys.TRAIN
)
eval_dataset = input_fn(
    test_texts,
    test_labels,
    hparams['batch_size'],
    mode=tf.estimator.ModeKeys.EVAL
)

CPU times: user 5min 9s, sys: 45.2 s, total: 5min 54s
Wall time: 4min 16s


In [12]:
hparams = {'learning_rate':.001}

model = keras_estimator(
    model_dir='output_dir',
    config=None,
    learning_rate=hparams['learning_rate'],
    embedding_path=None,
    word_index=None
)

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_2 (Dropout)          (None, 50, 128)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 50, 64)            24640     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 16, 64)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 16, 128)           24704     
_________________________________________________________________
global_average_pooling1d_1 ( (None, 128)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                

In [13]:
%%time
tf.random.set_seed(SEED)
model.fit(
    train_dataset,
    epochs=1,
    validation_data=eval_dataset,
    validation_steps=None
)

564/564 [==============================] - 9s 16ms/step - loss: 0.6671 - accuracy: 0.7038 - precision_1: 0.7644 - recall_1: 0.6086 - val_loss: 0.5010 - val_accuracy: 0.7940 - val_precision_1: 0.8230 - val_recall_1: 0.7558
CPU times: user 12.9 s, sys: 2.87 s, total: 15.8 s
Wall time: 12.7 s


In [14]:
model.save('headline_classification_model.h5')

# Model Serving 

We need to do custom model serving. See notebook LINK to deploy